In [1]:
from keras.applications import inception_v3
from keras import backend as K
from keras.preprocessing import image

import scipy
import numpy as np
import cv2


Using TensorFlow backend.
/home/grigoriy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/grigoriy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/grigoriy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/grigoriy/.local/lib/python3.6/site-packages/tensorflo

In [2]:
# Constants
DEMO_IMAGE_PATH = './test_images/flower_valley.jpg'

In [3]:
# Loop's hyperparameters
STEP = 0.01          # Value of the gradient ascent step
NUM_OCTAVE = 3       # Octave amount
OCTAVE_SCALE = 1.4   # Scale of each octave (look at the picture at the end of notebook)
ITERATIONS = 20      # Iterations amount on the each octave
MAX_LOSS = 10.       # Max loss that prevent ugly artifacts

In [4]:
K.set_learning_phase(0) #Set learning phase = 0 (test mode). Prevent model learning for safety reasons.

In [5]:
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

In [6]:
# Layers could be from mixed0 up to mixed10
# Check names with command model.summary()
layer_contribution = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5
}

In [7]:
# Loss calculation (L2_norm of layers outputs)
layer_dict = dict([(layer.name, layer) for layer in model.layers])
loss = K.variable(0.)

for layer_name in layer_contribution:
    coeff = layer_contribution[layer_name]
    # Get layer output
    activation = layer_dict[layer_name].output
    # Calculate L2 norm
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation)) / scaling  #test without -2:2 and so on

### Setting up gradient ascent

In [8]:
## Create K.function to fetch loss and gradients from model input

# Create tensor for result storing
dream = model.input 
# Fetch gradient and normalize it
grads = K.gradients(loss = loss, variables = dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)  #1e-7 - safety measure to avoid division by 0
# And now we provide link between current result and his gradients with losses
fetch_loss_and_grads = K.function([dream], [loss, grads])

# Gradient ascent's helper function
def eval_loss_and_grads(x):
    '''
    Extract loss and grads values from current input state
    Input:
        Current state of model.input
    Output:
        Current values of loss and input gradients
    '''
    # Pass input x (it will be model.input) through  K.function([model.input], [loss, grads])
    outs = fetch_loss_and_grads([x])
    # Get values
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values
    
# Main function
def gradient_ascent(x, iterations, step, max_loss=None, verbose=True):
    '''
    Performs the specified number of gradient ascent steps.
    '''
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if verbose:
            print('Loss value at {} step: {:.f4}'.format(i, loss_value))
        if max_loss is not None and loss_value > max_loss:
            print('Current loss = {} exceeded max_loss = {}, ascent was finished'.format(loss_value, max_loss))
            break
        x += step * grad_values
    return x

### Deep Dream main loop

In [9]:
# Base helper functions

def preprocess_img(img_path):
    '''
    Convert raw image to the InceptionV3 input format
    Input:
        Path to file, str
    Output:
        Numpy arrray with shape [1, img_height, img_width, 3]
    '''
    img = image.load_img(img_path)
    img = image.image_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def resize_img(img, size):
    '''
    Resize input image by zooming it
    Input:
        Numpy matrix with shape [1, img_height, img_width, 3]
    Output:
        Numpy matrix with shape [1, zoomed_img_height, zoomed_img_width, 3]
    '''
    img = np.copy(img)
    factor = (1, float(size[0])/img.shape[1], float(size[1])/img.shape[2], 1)
    return scipy.ndimage.zoom(img, factor, order=1) #order - spline interpolation order. Could be in range(0, 5)

def deprocess_img(x):
    '''
    Convert InceptionV3's result input after gradient ascent to the np.array image
    Input:
        Numpy arrray with shape [1, img_height, img_width, 3]
    Output:
        Numpy arrray with shape [img_height, img_width, 3]
    '''
    if K.image_data_format() == 'channel_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape(x.shape[1], x.shape[2], 3)  
    # Undo InceptionV3 preprocess input
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x
    

def save_img(img, file_name):
    '''
    Save image into png format
    Input:
        Numpy arrray with shape [1, img_height, img_width, 3]
    Output:
        None
    '''
    pil_img = deprocess_img(np.copy(img))
    cv2.imwrite(file_name, pil_img)

## Add img1

In [13]:
# # Loop's hyperparameters
# STEP = 0.01          # Value of the gradient ascent step
# NUM_OCTAVE = 3       # Octave amount
# OCTAVE_SCALE = 1.4   # Scale of each octave (look at the picture at the end of notebook)
# ITERATIONS = 20      # Iterations amount on the each octave
# MAX_LOSS = 10.       # Max loss that prevent ugly artifacts

In [14]:
img = preprocess_img(DEMO_IMAGE_PATH)

# Create list of shapes correspond with octave scales 
original_shape = img.shape[1:3]
octave_shapes = [original_shape]
for i in range(NUM_OCTAVE):
    scaled_shape = (original_shape/(OCTAVE_SCALE ** i)).astype(int)
    octave_shapes.append(scaled_shape)
octave_shapes = octave_shapes[::-1]


orginal_img = np.copy(img)

# Initialize shrunck image by the smallest image
shrunck_original_img = resize_img(img, octave_shapes[0]) 

for shape in octave_shapes:
    print('Processing image shape: ', shape)
    # Image gradient ascenting 
    img = resize_img(img, shape)
    img = gradient_ascent(img, ITERATIONS, STEP, MAX_LOSS)
    # Lost detail computation
    upscaled_shrunck_original_img = resize_img(shrunck_original_img, shape)
    same_original_size = reize_img(orginal_img, shape)
    lost_detail = same_original_size - upscaled_shrunck_original_img
    # Impute details
    img += lost_detail
    save_img(img, './notebook_images/scale_{}.png'.format(shape))
    
    shrunck_original_img = resize_img(orginal_img, shape)

save_img(img, 'result.png')    
    


AttributeError: 'NoneType' object has no attribute 'shape'